# Reading Wind Data from NAM Files
- North American Mesoscale Forecast System (NAM) [12 km]: The North American Mesoscale Forecast System (NAM) is one of the major regional weather forecast models run by the National Centers for Environmental Prediction (NCEP) for producing weather forecasts. Dozens of weather parameters are available from the NAM grids, from temperature and precipitation to lightning and turbulent kinetic energy. The NAM generates multiple grids (or domains) of weather forecasts over the North American continent at various horizontal resolutions. High-resolution forecasts are generated within the NAM using additional numerical weather models. These high-resolution forecast windows are generated over fixed regions and are occasionally run to follow significant weather events, like hurricanes. This dataset contains a 12 km horizontal resolution Lambert Conformal grid covering the Continental United States (CONUS) domain. It is run four times daily at 00z, 06z, 12z and 18z out to 84 hours with a 1 hour temporal resolution.
- Since the 0-hour forecast is the initial state of the model, it should match the analysis because it's based on the same observed data. Differences, if any, are generally minimal and result from the interpolation process in the model.
- https://www.ncei.noaa.gov/products/weather-climate-models/north-american-mesoscale
- https://www.ncei.noaa.gov/has/HAS.FileAppRouter?datasetname=NAMANL218&subqueryby=STATION&applname=&outdest=FILE

In [63]:
# Import modules
import pygrib
import numpy as np
import pandas as pd
import os
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import tarfile

# First, get .tar files from web download, extract the .grb2 files

In [78]:
# URL of the page with file listings
base_url = 'https://www.ncei.noaa.gov/pub/has/model/HAS012561864/'

# Fetch the webpage
response = requests.get(base_url)

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Directory to save the files
download_dir = 'downloaded_files'
os.makedirs(download_dir, exist_ok=True)
# Directory to extract the contents
extract_dir = 'extracted_files'
os.makedirs(extract_dir, exist_ok=True)

# Find all the links to .tar files
for link in soup.find_all('a', href=True):
    file_url = link['href']
    if file_url.endswith('.tar'):
        full_url = base_url + file_url
        file_name = os.path.join(download_dir, file_url.split('/')[-1])
        print(file_name)
        # Download the file
        print(f'Downloading {full_url}')
     #   file_response = requests.get(full_url)   # uncomment these lines to download them all - takes awhile!
     #   with open(file_name, 'wb') as file:
     #       file.write(file_response.content)
     # Open and extract the tar file
        print(f'Downloaded: {file_name}')
        extract_folder = extract_dir+'/'+file_url.strip('.tar')
        os.makedirs(extract_folder, exist_ok=True)
     #   with tarfile.open(file_name, 'r') as tar:
     #       tar.extractall(path=extract_folder)
        print(f'Extracted: {file_name}')
     #   os.remove(file_name) # delete file permanently
        print(f'Deleted: {file_name}')
        

downloaded_files/nam_218_2024020812.g2.tar
Downloaded: downloaded_files/nam_218_2024020812.g2.tar
Extracted: downloaded_files/nam_218_2024020812.g2.tar
Deleted: downloaded_files/nam_218_2024020812.g2.tar
downloaded_files/nam_218_2024020818.g2.tar
Downloaded: downloaded_files/nam_218_2024020818.g2.tar
Extracted: downloaded_files/nam_218_2024020818.g2.tar
Deleted: downloaded_files/nam_218_2024020818.g2.tar
downloaded_files/nam_218_2024020912.g2.tar
Downloaded: downloaded_files/nam_218_2024020912.g2.tar
Extracted: downloaded_files/nam_218_2024020912.g2.tar
Deleted: downloaded_files/nam_218_2024020912.g2.tar
downloaded_files/nam_218_2024020918.g2.tar
Downloaded: downloaded_files/nam_218_2024020918.g2.tar
Extracted: downloaded_files/nam_218_2024020918.g2.tar
Deleted: downloaded_files/nam_218_2024020918.g2.tar
downloaded_files/nam_218_2024021012.g2.tar
Downloaded: downloaded_files/nam_218_2024021012.g2.tar
Extracted: downloaded_files/nam_218_2024021012.g2.tar
Deleted: downloaded_files/nam_21

# Next, extract the needed values (wind speed, wind direction, temp, kinetic energy, etc)

In [ ]:
# Coordinates of anemometer
location_lat = 47.8437031
location_lon = -102.8524607

In [52]:
# Investigate how to dig into NAM data

# Confirm that unit of wind components is m/s and boundary layer level 0 (100-1000 m off ground)
grbs = pygrib.open('nam_files/nam_218_2024020806.g2/nam_218_20240208_0600_002.grb2')

# These are the desired data 
u_grb = grbs.select(name='U component of wind', level=0)[0]  # Select U component
v_grb = grbs.select(name='V component of wind', level=0)[0]  # Select V component
vert_vel_grb = grbs.select(name='Geometric vertical velocity', level=925)[0]  # 100000 Pa = about ground level
temp_K = grbs.select(name='Temperature', level=0)[0] # temp surface level, in K
kinetic_e = grbs.select(name='Turbulent kinetic energy', level=925)[0] # Turbulent kinetic energy:J kg**-1

print(u_grb)
print(v_grb)
print(vert_vel_grb)
print(temp_K)
print(kinetic_e)

for grb in grbs:
   #print(grb)
   if (grb.name == "Turbulent kinetic energy"):
        print(grb)
        print(grb.values[0][0])

grbs.close()

8:U component of wind:m s**-1 (instant):lambert:planetaryBoundaryLayer:level 0:fcst time 2 hrs:from 202402080600
9:V component of wind:m s**-1 (instant):lambert:planetaryBoundaryLayer:level 0:fcst time 2 hrs:from 202402080600
304:Geometric vertical velocity:m s**-1 (instant):lambert:isobaricInhPa:level 92500 Pa:fcst time 2 hrs:from 202402080600
339:Temperature:K (instant):lambert:surface:level 0:fcst time 2 hrs:from 202402080600
307:Turbulent kinetic energy:J kg**-1 (instant):lambert:isobaricInhPa:level 92500 Pa:fcst time 2 hrs:from 202402080600
23:Turbulent kinetic energy:J kg**-1 (instant):lambert:isobaricInhPa:level 5000 Pa:fcst time 2 hrs:from 202402080600
0.020000000330000007
31:Turbulent kinetic energy:J kg**-1 (instant):lambert:isobaricInhPa:level 7500 Pa:fcst time 2 hrs:from 202402080600
0.019999995422363283
39:Turbulent kinetic energy:J kg**-1 (instant):lambert:isobaricInhPa:level 10000 Pa:fcst time 2 hrs:from 202402080600
0.019999995422363283
47:Turbulent kinetic energy:J kg*

In [55]:
# Define function to read in file and compute wind speed and direction closest to an input lat and long

def get_wind(filename, lat, lon):
    
    # open file
    grbs = pygrib.open(filename)

    # Extract U and V components, and wind speed
    u_grb = grbs.select(name='U component of wind', level=80)[0]  # Select U component, 80m above ground
    v_grb = grbs.select(name='V component of wind', level=80)[0]  # Select V component, 80m above ground
    vert_vel_grb = grbs.select(name='Geometric vertical velocity', level=925)[0]  # 100000 Pa about ground level
    temp_K = grbs.select(name='Temperature', level=0)[0] # temp surface level, in K
    kinetic_e = grbs.select(name='Turbulent kinetic energy', level=925)[0] # Turbulent kinetic energy:J kg**-1

    # Get data and lat/lon
    u_data = u_grb.values
    v_data = v_grb.values
    vert_vel_data = vert_vel_grb.values
    temp_data = temp_K.values
    kinetic_e_data = kinetic_e.values
    lats, lons = u_grb.latlons()

    # Find the closest grid point
    from scipy.spatial import cKDTree
    tree = cKDTree(list(zip(lats.flatten(), lons.flatten())))
    dist, index = tree.query((lat, lon))

    # Get the u and v components at the closest grid point
    u_value = u_data.flatten()[index] # U Component: Represents wind speed in the east-west direction (positive values indicate winds from west).
    v_value = v_data.flatten()[index] # V Component: Represents wind speed in the north-south direction (positive values winds from south).
    vert_vel_value = vert_vel_data.flatten()[index] # Represents vetical component of wind speed
    temp_value = temp_data.flatten()[index] - 273.15 # temp in Celcius
    kinetic_e_value = kinetic_e_data.flatten()[index]

    # Calculate horizontal wind speed at the closest grid point
    wind_speed_value = np.sqrt(u_value**2 + v_value**2)

    welv_value = np.arctan(vert_vel_value/wind_speed_value) * 180 / np.pi

    # Calculate wind direction
    # Calculation: The wind direction is calculated using the arctan2 function which provides the angle in radians. Convert this angle to degrees and adjust it to the 0-360 degree range.
    # Result: The result is the wind direction in degrees where 0° represents wind coming from the north, 90° from the east, 180° from the south, and 270° from the west.
    wind_direction = (np.arctan2(-u_value, -v_value) * 180 / np.pi) % 360

    datetime_str = filename[40:48]+filename[49:53]
    datetime_obj = datetime.strptime(datetime_str, '%Y%m%d%H%M')
    datetime_obj = pd.to_datetime(datetime_obj)
    forecast_str = filename[54:57]

    grbs.close()

    return datetime_obj, forecast_str, wind_speed_value, wind_direction, welv_value, temp_value, kinetic_e_value  # utc time, m/s, and degrees

# Try it out
get_wind('nam_files/nam_218_2024020806.g2/nam_218_20240208_0600_002.grb2', location_lat, location_lon )



(Timestamp('2024-02-08 06:00:00'),
 '002',
 6.2837645855587185,
 330.3254928993131,
 -0.06953458131239801,
 -1.0156054687499818,
 1.2000000000000002)

In [6]:
# Check that wind direction calculation works as expected
u = -1
v = 0
wind_direction_demo = (np.arctan2(u,v) * 180 / np.pi) % 360
wind_direction_demo

270.0

In [56]:
# Loop through NAM data and get desired data

# Define the directory path and folders inside
directory_path = 'nam_files'  # change this to 'extracted_files' if the files are downloaded using the script (and add 6 to indices in for loop)
folder_names = sorted(os.listdir(directory_path)) 

# To save harddrive space and computation time, just keep these forecast periods
forecasts_to_keep = [0,1,2,3,4,6,12,24,48,72]

# Empty list to store wind data
nam_wind_data = []

# Loop through all files in the directory, and find wind data for each date
counter = 1
counter_data_added = 0
for folder_name in folder_names:
    if folder_name != '.DS_Store':
        # Create full path to the file
        folder_path = os.path.join(directory_path, folder_name)
        file_names = sorted(os.listdir(folder_path))
        for file_name in file_names: 
            if file_name != '.DS_Store':
                forecast = int(file_name[22:25])
                date = file_name[8:16]
                time = file_name[17:21]
                file_path = os.path.join(folder_path, file_name)
                if forecast not in forecasts_to_keep:
                    os.remove(file_path) # delete file permanently
                    print(f"{counter}: date {date}, time {time}, forecast period {forecast}: file deleted")
                else:
                    try:
                        nam_wind_data.append(get_wind(file_path, location_lat, location_lon )) 
                        print(f"{counter}: date {date}, time {time}, forecast period {forecast}: data added!")
                        counter_data_added+=1
                    except:
                        print(f"{counter}: date {date}, time {time}, forecast period {forecast}: no data found")
                counter+=1
print(f"Done! {counter_data_added} files successfully added.")


1: date 20240208, time 0000, forecast period 0: data added!
2: date 20240208, time 0000, forecast period 1: data added!
3: date 20240208, time 0000, forecast period 2: data added!
4: date 20240208, time 0000, forecast period 3: data added!
5: date 20240208, time 0000, forecast period 4: data added!
6: date 20240208, time 0000, forecast period 6: data added!
7: date 20240208, time 0000, forecast period 12: data added!
8: date 20240208, time 0000, forecast period 24: data added!
9: date 20240208, time 0000, forecast period 48: data added!
10: date 20240208, time 0000, forecast period 72: data added!
11: date 20240208, time 0600, forecast period 0: data added!
12: date 20240208, time 0600, forecast period 1: data added!
13: date 20240208, time 0600, forecast period 2: data added!
14: date 20240208, time 0600, forecast period 3: data added!
15: date 20240208, time 0600, forecast period 4: data added!
16: date 20240208, time 0600, forecast period 6: data added!
17: date 20240208, time 0600,

In [57]:
# Crate dataframe for nam data
nam_wind_data_df = pd.DataFrame(nam_wind_data, columns=['time_utc','forecast_period','wspd','wdr','welv','temp_C','kinetic_e'])
nam_wind_data_df = nam_wind_data_df.sort_values(by=['time_utc', 'forecast_period'])
nam_wind_data_df

,time_utc,forecast_period,wspd,wdr,welv,temp_C,kinetic_e
0,2024-02-08 00:00:00,000,4.856055,53.385763,-0.148575,-0.297461,0.6
1,2024-02-08 00:00:00,001,4.419276,45.718757,0.144892,-0.441577,0.5
2,2024-02-08 00:00:00,002,3.942619,36.174609,-0.187564,-0.437842,0.4
3,2024-02-08 00:00:00,003,4.133296,24.822946,0.279599,-0.465015,0.4
4,2024-02-08 00:00:00,004,4.535726,15.952962,-0.101502,-0.503418,0.5
...,...,...,...,...,...,...,...
1313,2024-03-11 18:00:00,006,6.680576,275.817073,-0.028047,1.159805,0.3
1314,2024-03-11 18:00:00,012,6.595477,281.342438,-0.183030,-2.321797,0.1
1315,2024-03-11 18:00:00,024,5.135105,143.975032,-0.115949,8.957837,1.1
1316,2024-03-11 18:00:00,048,3.347656,281.716584,0.175220,4.328857,0.6


# Finally, save the data into a csv file

In [58]:
# Save nam data into a csv
nam_wind_data_df.to_csv('nam_wind_data.csv', index=False)